###**This notebook is deprecated and will no longer be updated.**

###**[The official notebook (made by oobabooga) is located here](https://colab.research.google.com/github/oobabooga/text-generation-webui/blob/main/Colab-TextGen-GPU.ipynb). Use it instead.**

####**Alternatively, try the [official KoboldCpp colab](https://colab.research.google.com/github/lostruins/koboldcpp/blob/concedo/colab.ipynb). It's fast, lightweight, and easy to use.**

In [ ]:
#@markdown
%%html
<b>👇 Press play on this music player to keep the current tab alive. Keep this page open and occasionally check for CAPTCHA's so that you are not disconnected.</b><br/><br />
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
import os, torch
from IPython.display import clear_output, display, HTML
from IPython.utils import capture
from google.colab import files, drive

#PARAMS
#@markdown 👈 Press this button after configuring the settings below to start the installation process. The links will appear at the bottom of this page after a few minutes.

Model = "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GPTQ" #@param ["https://huggingface.co/TheBloke/LLaMA2-13B-Tiefighter-GPTQ", "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GPTQ", "https://huggingface.co/TheBloke/Mistral-ClaudeLimaRP-v3-7B-GPTQ", "https://huggingface.co/TheBloke/Mythalion-13B-GPTQ", "https://huggingface.co/TheBloke/MythoMax-L2-13B-GPTQ",    "https://huggingface.co/TheBloke/MythoMist-7B-GPTQ", "https://huggingface.co/TheBloke/openchat_3.5-GPTQ", "https://huggingface.co/TheBloke/ReMM-SLERP-L2-13B-GPTQ", "https://huggingface.co/TheBloke/Toppy-M-7B-GPTQ",  "https://huggingface.co/TheBloke/Xwin-MLewd-13B-v0.2-GPTQ",   "https://huggingface.co/TheBloke/zephyr-7B-beta-GPTQ"]{allow-input: true}
#@markdown > <font color="gray">Enter any model URL from [Hugging Face](https://huggingface.co/models). Tap the arrow to view some examples.
Branch = "" #@param {type:"string"}
#@markdown > <font color="gray">You can specify a valid model branch here. Defaults to ``main`` if left blank.
Flags = "--share --api --public-api --chat-buttons" #@param {type:"string"}
#@markdown > <font color="gray">You can write any valid [command-line flags](https://github.com/oobabooga/text-generation-webui/blob/main/README.md#basic-settings) or [extensions](https://github.com/oobabooga/text-generation-webui/blob/main/docs/Extensions.md) here.
Google_Drive = False #@param {type:"boolean"}
#@markdown > <font color="gray">Remember that models are very large, and free Google Drive only provides 15GB. [Manage your Drive space here.](https://drive.google.com/drive/my-drive)
Debug = False #@param {type:"boolean"}
#@markdown > <font color="gray">Shows the installation console log if checked.

def message_a():
	clear_output(wait = True)
	print(f"\033[1;32;1m\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n\nBeginning installation. This will take a few minutes.\n\nInstalling WebUI...\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n\033[0;37;0m")
	display(HTML("<img src='https://i.redd.it/cf74oy8te80c1.gif' style='vertical-align:middle;margin:0px 50px'/>"))
def message_b():
	clear_output(wait = True)
	print(f"\033[1;32;1m\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n\nBeginning installation. This will take a few minutes.\n\nWebUI installed. Downloading model...\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n\033[0;37;0m")
	display(HTML("<img src='https://i.redd.it/cf74oy8te80c1.gif' style='vertical-align:middle;margin:0px 50px'/>"))
def message_c():
	clear_output(wait = True)
	print(f"\033[1;32;1m\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n\nInstallation complete. To enter the WebUI, click on the gradio link that will appear below.\n\nFor SillyTavern users, copy the Blocking API URL and paste into SillyTavern's API settings.\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n\033[0;37;0m")

# Install WebUI
def install_webui():
	if os.path.exists(repo_dir):
		%cd {repo_dir}
		!git pull
	else:
		!git clone https://github.com/oobabooga/text-generation-webui.git
	%cd {repo_dir}

	# Install WebUI requirements
	torver = torch.__version__
	is_cuda118 = '+cu118' in torver  # 2.1.0+cu118
	is_cuda117 = '+cu117' in torver  # 2.0.1+cu117
	textgen_requirements = open('requirements.txt').read().splitlines()
	if is_cuda117:
		textgen_requirements = [req.replace('+cu121', '+cu117').replace('+cu122', '+cu117').replace('torch2.1', 'torch2.0') for req in textgen_requirements]
	elif is_cuda118:
		textgen_requirements = [req.replace('+cu121', '+cu118').replace('+cu122', '+cu118') for req in textgen_requirements]
	with open('temp_requirements.txt', 'w') as file:
		file.write('\n'.join(textgen_requirements))
	!pip install -r temp_requirements.txt --upgrade

	# Install extension requirements
	if 'deepspeed' in Flags:
		!pip install -U mpi4py
		!pip install -U deepspeed
	if 'xformers' in Flags:
		!pip install xformers
	if 'api' in Flags:
		!pip install -r extensions/openai/requirements.txt
	if 'google_translate' in Flags:
		!pip install -r extensions/google_translate/requirements.txt
	if 'superbooga' in Flags:
		!pip install -r extensions/superbooga/requirements.txt
	if 'silero_tts' in Flags:
		!pip install -r extensions/silero_tts/requirements.txt
	if 'elevenlabs_tts' in Flags:
		!pip install -r extensions/elevenlabs_tts/requirements.txt
	if 'whisper_stt' in Flags:
		!pip install -r extensions/whisper_stt/requirements.txt
	if 'openai' in Flags:
		!pip install -r extensions/openai/requirements.txt
	if 'ngrok' in Flags:
		!pip install -r extensions/ngrok/requirements.txt

	try:
		import flash_attn
	except:
		!pip uninstall -y flash_attn

# Mount Google Drive if enabled
if Google_Drive:
	drive.mount('/content/drive')
	%cd /content/drive/MyDrive
	repo_dir = '/content/drive/MyDrive/text-generation-webui'
	model_dir = '/content/drive/MyDrive/text-generation-webui/models'
else:
	%cd /content
	repo_dir = '/content/text-generation-webui'
	model_dir = '/content/text-generation-webui/models'
if not Debug:
	message_a()
	with capture.capture_output() as cap:
		install_webui()
else:
	install_webui()

# Download model
def install_model():
	global output_folder
	model_url = Model.strip()
	if model_url != "":
		if not model_url.startswith('http'):
			model_url = 'https://huggingface.co/' + model_url
		url_parts = model_url.strip('/').strip().split('/')
		output_folder = f"{url_parts[-2]}_{url_parts[-1]}"
		branch = Branch.strip('"\' ')
		if Branch.strip() not in ['', 'main']:
			output_folder += f"_{branch}"
			!python download-model.py {model_url} --branch {branch}
		else:
			!python download-model.py {model_url}
	else:
		output_folder = ""

if not Debug:
	message_b()
	with capture.capture_output() as cap:
		install_model()
else:
	install_model()

# Run WebUI
if not Debug:
	message_c()
if 'deepspeed' in Flags:
	cmd = f"deepspeed --num_gpus=1 server.py"
else:
	cmd = f"HF_TOKEN='hf_xxxxxxxxxxxxxx' python server.py"
if output_folder != "":
    cmd += f" --model {output_folder}"
cmd += f" {Flags}"
print(cmd)
!$cmd